<a href="https://colab.research.google.com/github/JTEC-Iritani/ICT-Training-2021/blob/main/python/Practice_14_Kuching_Traffic_FBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kuching Traffic Data

In [1]:
# Import the libraries 使用するライブラリのインポート
import pandas as pd
import numpy as np
import pip
import datetime as dt
from fbprophet import Prophet
import matplotlib.pyplot as plt #グラフ用
# import openlocationcode
# !pip openlocationcode
import folium #地図作成用
from folium.plugins import HeatMap # ヒートマップ用ライブラリ

In [2]:
# ローカルファイル（クチン市交通量データ）をアップロードする（Google Colaboratoryの場合）
# The Littlest JupyterHubの場合：下記の2行は不要。別途マニュアルでCSVファイルをアップロードしておく
from google.colab import files
uploaded = files.upload()

Saving Kuching_Traffic_Data_202003_1.pickle to Kuching_Traffic_Data_202003_1.pickle


In [3]:
# pickleデータをDataFrameに変換（フォルダ名を必要に応じ修正すること）
df = pd.read_pickle('../data/Kuching_Traffic_Data_202003_1.pickle')
# df = pd.read_pickle('Kuching_Traffic_Data_202003_1.pickle') # Google Colaboratory用
# データの内容を表示
df
# df.head(3)

,ds,y,INT_ID
0,2020-03-01 00:05:00,32,8207
1,2020-03-01 00:10:00,24,8207
2,2020-03-01 00:15:00,29,8207
3,2020-03-01 00:20:00,28,8207
4,2020-03-01 00:25:00,27,8207
...,...,...,...
14680,2020-03-17 23:40:00,20,8226
14681,2020-03-17 23:45:00,17,8226
14682,2020-03-17 23:50:00,17,8226
14683,2020-03-17 23:55:00,28,8226


In [4]:
df.info()
# df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14685 entries, 0 to 14684
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      14685 non-null  datetime64[ns]
 1   y       14685 non-null  int64         
 2   INT_ID  14685 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 344.3 KB


In [5]:
INT_List = df.INT_ID.unique()
INT_List

array([8207, 8208, 8226])

In [6]:
df1 = df[df['INT_ID'] == 8207] #交差点IDを指定してデータを抽出
df1

,ds,y,INT_ID
0,2020-03-01 00:05:00,32,8207
1,2020-03-01 00:10:00,24,8207
2,2020-03-01 00:15:00,29,8207
3,2020-03-01 00:20:00,28,8207
4,2020-03-01 00:25:00,27,8207
...,...,...,...
4890,2020-03-17 23:40:00,21,8207
4891,2020-03-17 23:45:00,23,8207
4892,2020-03-17 23:50:00,16,8207
4893,2020-03-17 23:55:00,21,8207


In [ ]:
# df1.describe()
df1.info()

In [ ]:
# Sample Code: 1日単位より短い間隔のデータ
# m = Prophet(changepoint_prior_scale=0.01).fit(df)
# future = m.make_future_dataframe(periods=300, freq='H')
# fcst = m.predict(future)
# fig = m.plot(fcst)

In [ ]:
#Python
# m = Prophet().fit(df1)
m = Prophet(changepoint_prior_scale=0.01).fit(df1)

In [ ]:
# Python
future = m.make_future_dataframe(periods=168, freq='H')
# future = m.make_future_dataframe(periods=365)

future.tail()

In [ ]:
# Python
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# Python
fig1 = m.plot(forecast)

In [ ]:
# Python
fig2 = m.plot_components(forecast)

In [7]:
# Google Colabの場合：ローカルファイル（クチン市交差点位置情報データ）をアップロードする（ファイル名：Kuching Intersections GPS.csv）
# The Littlest JupyterHubの場合：下記の2行は不要。別途マニュアルでCSVファイルをアップロードしておく
from google.colab import files
uploaded = files.upload()

Saving Kuching Intersections GPS.csv to Kuching Intersections GPS.csv


In [8]:
# CSVをDataFrameに変換して表示（フォルダ名を必要に応じ修正すること）
df_kuching = pd.read_csv('../data/Kuching Intersections GPS.csv')
# df_kuching = pd.read_csv('Kuching Intersections GPS.csv') # Google Colaboratory用

df_kuching.head(8)

,SEQ,INT_ID,LAT,LNG,INT_NAME
0,1,8204,1.541509,110.371257,Jalan Foochow No.1 / Tun Razak Junction (Kuchi...
1,2,8238,1.520550,110.334121,Jalan Batu Kawa / Sunny Hill
2,3,8251,1.509839,110.301923,Jalan Batu Kawa / Taman Desa Wira
3,4,8255,1.521340,110.354219,Jalan Song / Laksamana Cheng Ho / Tun Razak / ...
4,5,8263,1.507804,110.292645,Jalan Batu Kawa / Jalan Stephen Yong
5,6,8280,1.556044,110.382255,Jalan Datuk Marican Salleh / Jalan Utama / Jal...
6,7,8285,1.514472,110.316484,Jalan Batu Kawa / Jalan Sungai Maong Utama
7,8,8295,1.489442,110.342426,Jalan Lapangan Terbang Antarabangsa Kuching (1...


In [10]:
# 地図オブジェクト作成
map_kuching = folium.Map(location=[1.523833, 110.347119], zoom_start=12)

# マーカープロット（測定交差点は青、それ以外は赤）
for i, row in df_kuching.iterrows():
    if (row['INT_ID'] in INT_List):
    # if (row['INT_ID'] == 8207) | (row['INT_ID'] == 8208) | (row['INT_ID'] == 8226):
        folium.Marker(
            location=[row['LAT'], row['LNG']],
            popup=str(row['INT_ID']) + ' ' + row['INT_NAME'],
            icon=folium.Icon(color='blue')
        ).add_to(map_kuching)
    else:
        folium.Marker(
            location=[row['LAT'], row['LNG']],
            # popup=row['INT_NAME'],
            popup=str(row['INT_ID']) + ' ' + row['INT_NAME'],
            icon=folium.Icon(color='red')
        ).add_to(map_kuching)
  
  # 地図表示
map_kuching